<h1 >CNN on CIFR </h1>

1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use Dense Layers (also called fully connected layers), or DropOut.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [1]:
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler

Using TensorFlow backend.


In [2]:
# Hyperparameters
batch_size = 64
l = 5
num_filter =40

epochs = 100
num_classes = 10
compression = 1
dropout_rate = 0

In [3]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170500096/170498071 [==============================] - 6s 0us/step


<h1> Image preprocessing </h1>

In [4]:
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True)
datagen.fit(X_train)
train_data=datagen.flow(X_train,y_train, batch_size=batch_size)
val_data=datagen.flow(X_test,y_test, batch_size=batch_size)

In [5]:
# Dense Block
def denseblock(input, num_filter, dropout_rate):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)                         
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        temp = concat
    return temp

## transition Blosck
def transition(input, num_filter,dropout_rate ):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(1,1))(relu)
    output = layers.Conv2D(num_classes,kernel_size=[4,4],strides=(1,1), activation='softmax')(AvgPooling)
    flat_output = layers.Flatten()(output)
    return flat_output

In [6]:
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [7]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 40)   1080        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 40)   160         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 40)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])

In [11]:
model.fit(train_data,batch_size=batch_size,epochs=120,verbose=1,validation_data=val_data)

Epoch 1/120
782/782 [==============================] - 39s 50ms/step - loss: 1.4061 - accuracy: 0.5072 - val_loss: 1.9530 - val_accuracy: 0.5024
Epoch 2/120
782/782 [==============================] - 38s 48ms/step - loss: 0.9944 - accuracy: 0.6510 - val_loss: 1.2105 - val_accuracy: 0.6008
Epoch 3/120
782/782 [==============================] - 38s 48ms/step - loss: 0.8126 - accuracy: 0.7151 - val_loss: 1.2512 - val_accuracy: 0.5909
Epoch 4/120
782/782 [==============================] - 38s 48ms/step - loss: 0.7098 - accuracy: 0.7525 - val_loss: 0.8436 - val_accuracy: 0.7061
Epoch 5/120
782/782 [==============================] - 38s 48ms/step - loss: 0.6376 - accuracy: 0.7766 - val_loss: 0.8907 - val_accuracy: 0.7092
Epoch 6/120
782/782 [==============================] - 38s 48ms/step - loss: 0.5822 - accuracy: 0.7983 - val_loss: 0.7485 - val_accuracy: 0.7471
Epoch 7/120
782/782 [==============================] - 37s 48ms/step - loss: 0.5377 - accuracy: 0.8141 - val_loss: 0.6789 - val_ac

Observation:


From above the model is trained for 120 epoch the highest test accuracy i got is around 0.9026 at 111th epoch this is accuracy which matches what an expected accuracy

when i train for more epoch the test accuaracy will be constant more than 90+ test accuracy